preliminary testing of bbox dataset

In [ ]:
# Mounting google drive

from google.colab import drive
drive.mount('/content/drive')

ROOT = "/content/drive/MyDrive/pothole_project"

Mounted at /content/drive


In [ ]:
# import statements

import xml.etree.ElementTree as ET
from pathlib import Path
import cv2, json, os
import random
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# setting directories

IM_DIR   = Path(ROOT)/"data_raw/bbox_data/images"
VOC_DIR  = Path(ROOT)/"data_raw/bbox_data/annotations"
YOLO_DIR = Path(ROOT)/"data_raw/bbox_data/labels"
YOLO_DIR.mkdir(exist_ok=True)

In [ ]:
# Helper to get image size robustly
def get_img_size(img_path):
    img = cv2.imread(str(img_path))
    if img is None:
        raise ValueError(f"Cannot read image: {img_path}")
    h, w = img.shape[:2]
    return w, h

In [ ]:
n_files = 0
for xml_path in VOC_DIR.glob("*.xml"):
    tree = ET.parse(str(xml_path))
    root = tree.getroot()

    # Find corresponding image file (dataset names vary; adjust extension if needed)
    fname = root.findtext("filename")
    img_path = IM_DIR/fname
    if not img_path.exists():
        # try to locate by changing extension if needed
        cand = list(IM_DIR.glob(Path(fname).stem + ".*"))
        if not cand:
            continue
        img_path = cand[0]

    W, H = get_img_size(img_path)

    yolo_lines = []
    for obj in root.findall("object"):
        cls = obj.findtext("name").strip().lower()
        if cls != "pothole":   # ignore any non-target classes if present
            continue
        bnd = obj.find("bndbox")
        xmin = float(bnd.findtext("xmin"))
        ymin = float(bnd.findtext("ymin"))
        xmax = float(bnd.findtext("xmax"))
        ymax = float(bnd.findtext("ymax"))

        # Convert to YOLO normalized cx, cy, w, h
        cx = ((xmin + xmax) / 2.0) / W
        cy = ((ymin + ymax) / 2.0) / H
        w  = (xmax - xmin) / W
        h  = (ymax - ymin) / H

        # Clip to [0,1] just in case of edge cases
        cx = min(max(cx, 0.0), 1.0)
        cy = min(max(cy, 0.0), 1.0)
        w  = min(max(w,  0.0), 1.0)
        h  = min(max(h,  0.0), 1.0)

        yolo_lines.append(f"0 {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

    # Write label file (same stem as image, .txt)
    if yolo_lines:
        (YOLO_DIR / (img_path.stem + ".txt")).write_text("\n".join(yolo_lines))
    n_files += 1

print("Converted XMLs:", n_files)

Converted XMLs: 665


In [ ]:
SPLIT_DIR= Path(ROOT)/"data_raw/bbox_data/splits"
SPLIT_DIR.mkdir(exist_ok=True)

In [ ]:
images = sorted([p for p in IM_DIR.glob("*.png") if (YOLO_DIR/(p.stem+".txt")).exists()])
random.Random(42).shuffle(images)

n = len(images)
n_train = int(0.7*n)
n_val   = int(0.2*n)
train = images[:n_train]
val   = images[n_train:n_train+n_val]
test  = images[n_train+n_val:]

def write_list(name, items):
    (SPLIT_DIR/f"{name}.txt").write_text("\n".join(str(p) for p in items))

write_list("train", train)
write_list("val",   val)
write_list("test",  test)
len(train), len(val), len(test)

(465, 133, 67)

In [ ]:
model = YOLO("yolov8n.pt")         # load pretrained model

In [ ]:
model.train(
    data="/content/drive/MyDrive/pothole_project/data_raw/bbox_data/dataset.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
)

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/pothole_project/data_raw/bbox_data/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a60641f2090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 